# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([165])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Fetch Dataset

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

# Import dataset

In [4]:
examples = mnist.data
classes = mnist.target

x = []
y = []
for (example, label) in zip(examples, classes):
    if label == "0":
        x.append(example)
        y.append(-1)
    elif label == "1":
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])
    ArbitraryUnitary(params[15:30], wires=[3, 4])
    ArbitraryUnitary(params[30:45], wires=[5, 6])

    # Second layer
    ArbitraryUnitary(params[45:60], wires=[0, 1])
    ArbitraryUnitary(params[60:75], wires=[2, 3])
    ArbitraryUnitary(params[75:90], wires=[4, 5])
    ArbitraryUnitary(params[90:105], wires=[6, 7])

    # Third layer
    ArbitraryUnitary(params[105:120], wires=[2, 5])

    # Fourth layer
    ArbitraryUnitary(params[120:135], wires=[1, 2])
    ArbitraryUnitary(params[135:150], wires=[5, 6])

    # Fifth layer
    ArbitraryUnitary(params[150:165], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802
 0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
 0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033
 0.38867729 0.27134903 0.82873751 0.35675333 0.28093451 0.54269608
 0.14092422 0.80219698 0.07455064 0.98688694 0.77224477 0.19871568
 0.00552212 0.81546143 0.70685734 0.72900717 0.77127035 0.07404465
 0.35846573 0.11586906 0.86310343 0.62329813 0.33089802 0.06355835
 0.31098232 0.32518332 0.72960618 0.6375574

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5802047781569966
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.8668941979522184
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.8725824800910125
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.8668941979522184
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.8964732650739476
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.9169510807736063
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.9510807736063709
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.9124004550625711
epoch 7

epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.9192263936291241
test_accuracies: [0.9510807736063709, 0.9124004550625711, 0.9522184300341296, 0.9408418657565415, 0.9249146757679181, 0.9203640500568828, 0.8998862343572241, 0.9192263936291241, 0.9419795221843004, 0.9340159271899886, 0.9476678043230944, 0.9215017064846416, 0.9306029579067122, 0.9408418657565415, 0.9328782707622298, 0.9249146757679181, 0.9283276450511946, 0.9249146757679181, 0.9283276450511946, 0.9385665529010239, 0.9169510807736063, 0.9249146757679181, 0.9306029579067122, 0.9374288964732651, 0.9294653014789533, 0.9226393629124005, 0.9271899886234357, 0.9306029579067122, 0.9158134243458476, 0.9192263936291241]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.9419795221843004
test_accuracies: [0.9124004550625711, 0.9522184300341296, 0.9408418657565415, 0.9249146757679181, 0.9203640500568828, 0.8998862343572241, 0.9192

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.66611809  1.12728912  0.17380951  0.49826847  0.42114932 -0.0933839
  0.2743382   0.22628943  1.011862    0.63936315 -0.15892487  0.7737641
  1.00278768  0.08135251  0.12161018 -0.16960133  0.41480442  0.75697046
  1.04507461  0.50933659  0.18077247  0.40580662 -0.0199781   0.91378977
  0.69126513  0.15922356  0.08327296  0.08285062  0.56440511 -0.39070958
  1.28286297 -0.11450666 -0.16509153  0.65598074  0.97769738  1.30684867
  0.79719865 -0.1318126   0.79558647  1.42455349  0.41491218  0.44635471
 -0.25068616  1.56548842  0.65581566  0.93190006  0.54640049  0.21229286
 -0.13089667 -0.16302513  1.20651272  0.48665411  1.51486551  0.89613883
  0.63278562 -0.08804016 -0.36722395  0.10026198 -0.43719453  0.61622507
  0.51889307  0.38595708  0.76518845  0.43068845  0.13034952  0.70052317
 -0.02871371  1.56817174 -0.2150989   1.02326916  0.93088746 -0.29374188
  0.13841299  0.08605693  0.70685734  0.96171981  1.19550922 -0.32939054
  0.41728409  0.30597519  

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.9481281010374379


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 70.08598852157593
training time: 11740.53304696083
test time: 516.2474353313446
total time: 12326.866470813751
